In [1]:
# import libraries
import requests
import pandas as pd
import os


In [2]:
# extract F1 data from API
response1 = requests.get('https://api.openf1.org/v1/drivers')
f1_drivers_api = response1.json()
response2 = requests.get('https://api.openf1.org/v1/meetings')
f1_meetings_api = response2.json()
response3 = requests.get('https://api.openf1.org/v1/weather')
f1_weather_api = response3.json()

# response3 = requests.get('https://api.openf1.org/v1/position')
# f1_position_api = response3.json()


In [3]:
# convert JSON to DataFrame and then to CSV
df_drivers = pd.DataFrame(f1_drivers_api)
df_drivers.to_csv('Files/Sample data/f1_drivers.csv')
df_meetings = pd.DataFrame(f1_meetings_api)
df_meetings.to_csv('Files/Sample data/f1_meetings.csv')
df_weather = pd.DataFrame(f1_weather_api)
df_weather.to_csv('Files/Sample data/f1_weather.csv')
# df_position = pd.DataFrame(f1_position_api)
# df_position.to_csv('f1_position.csv')


In [4]:
# connect to local PostgreSQL
import psycopg2
conn = psycopg2.connect(
    dbname = 'f1_db',
    user = 'postgres',
    password = '123456Superman',
    host = 'localhost'
)
cur = conn.cursor()


In [5]:
# Create tables in PostgreSQL database
# Create "drivers" table
cur.execute("""
DROP TABLE IF EXISTS drivers;
            
CREATE TABLE drivers (
    index SERIAL PRIMARY KEY,
    meeting_key INT,
    session_key INT,
    driver_number INT,
    broadcast_name VARCHAR(50),
    full_name VARCHAR(50),
    name_acronym VARCHAR(20),
    team_name VARCHAR(20),
    team_colour VARCHAR(20),
    first_name VARCHAR(30),
    last_name VARCHAR(30),
    headshot_url VARCHAR(200),
    country_code VARCHAR(20),
    "registered" TIMESTAMP
);
""")

# Create "meetings" table
cur.execute("""
DROP TABLE IF EXISTS meetings;
            
CREATE TABLE meetings (
    index SERIAL PRIMARY KEY,
    meeting_key INT,
    meeting_name VARCHAR(50),
    meeting_official_name VARCHAR(200),
    location VARCHAR(50),
    country_key INT,
    country_code VARCHAR(50),
    country_name VARCHAR(50), 
    circuit_key INT,
    circuit_short_name VARCHAR(50),
    date_start TIMESTAMPTZ,
    gmt_offset VARCHAR(50), 
    year INT,
    meeting_code VARCHAR(20),
    "registered" TIMESTAMP
);
""")

# Create "weather" table
cur.execute("""
DROP TABLE IF EXISTS weather;

CREATE TABLE weather (
    index SERIAL PRIMARY KEY,
    meeting_key INT,
    session_key INT,
    air_temperature DECIMAL(5,2),
    humidity DECIMAL(5,2),
    pressure DECIMAL(6,2),
    rainfall DECIMAL(5,2),
    track_temperature DECIMAL(5,2),
    wind_direction INT,
    wind_speed DECIMAL(5,2),
    date TIMESTAMPTZ,
    "registered" TIMESTAMP
);
""")

In [6]:
# insert values into tables
# drivers table
for index, row in df_drivers.iterrows():
    cur.execute("""
        INSERT INTO drivers (
            meeting_key, session_key, driver_number, broadcast_name, full_name, 
            name_acronym, team_name, team_colour, first_name, last_name, 
            headshot_url, country_code, "registered"
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, NOW());
    """, (
        row['meeting_key'], row['session_key'], row['driver_number'], row['broadcast_name'], row['full_name'], 
        row['name_acronym'], row['team_name'], row['team_colour'], row['first_name'], 
        row['last_name'], row['headshot_url'], row['country_code']
    ))
    conn.commit()

# meetings table
for index, row in df_meetings.iterrows():
        cur.execute("""
            INSERT INTO meetings (
                meeting_key, meeting_name, meeting_official_name, location, 
                country_key, country_code, country_name, circuit_key, 
                circuit_short_name, date_start, gmt_offset, year, 
                meeting_code, "registered"
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, NOW());
        """, (
            row['meeting_key'], row['meeting_name'], row['meeting_official_name'], row['location'], 
            row['country_key'], row['country_code'], row['country_name'], row['circuit_key'], 
            row['circuit_short_name'], row['date_start'], row['gmt_offset'], row['year'], 
            row['meeting_code']
        ))
        conn.commit()


# weather table
for index, row in df_weather.iterrows():
        cur.execute("""
            INSERT INTO weather (
                meeting_key, session_key, air_temperature, humidity, pressure, 
                rainfall, track_temperature, wind_direction, wind_speed, date, "registered"
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, NOW());
        """, (
            row['meeting_key'], row['session_key'], row['air_temperature'], row['humidity'], 
            row['pressure'], row['rainfall'], row['track_temperature'], row['wind_direction'], 
            row['wind_speed'], row['date']
        ))
        conn.commit()

In [ ]:
# check values in table drivers
cur.execute("SELECT * FROM drivers;")
drivers_preview = pd.DataFrame(cur.fetchall())
print(drivers_preview)


        0     1     2   3             4               5    6   \
0        1  1140  7763   1  M VERSTAPPEN  Max VERSTAPPEN  VER   
1        2  1140  7763   2    L SARGEANT  Logan SARGEANT  SAR   
2        3  1140  7763   4      L NORRIS    Lando NORRIS  NOR   
3        4  1140  7763  10       P GASLY    Pierre GASLY  GAS   
4        5  1140  7763  11       S PEREZ    Sergio PEREZ  PER   
...    ...   ...   ...  ..           ...             ...  ...   
4834  4835  1255  9989  44    L HAMILTON  Lewis HAMILTON  HAM   
4835  4836  1255  9989  55       C SAINZ    Carlos SAINZ  SAI   
4836  4837  1255  9989  63     G RUSSELL  George RUSSELL  RUS   
4837  4838  1255  9989  81     O PIASTRI   Oscar PIASTRI  PIA   
4838  4839  1255  9989  87     O BEARMAN  Oliver BEARMAN  BEA   

                   7       8       9           10  \
0     Red Bull Racing  3671C6     Max  Verstappen   
1            Williams  37BEDD   Logan    Sargeant   
2             McLaren  F58020   Lando      Norris   
3      

In [8]:
# check values in table meetings
cur.execute("SELECT * FROM meetings;")
drivers_preview = pd.DataFrame(cur.fetchall(), columns=['index', 'meeting_key', 'meeting_name', 'meeting_official_name',
'location', 'country_key', 'country_code', 'country_name', 'circuit_key',
'circuit_short_name', 'date_start', 'gmt_offset', 'year', 'meeting_code', 'registered'])
print(drivers_preview)

    index  meeting_key               meeting_name  \
0       1         1140         Pre-Season Testing   
1       2         1141         Bahrain Grand Prix   
2       3         1142   Saudi Arabian Grand Prix   
3       4         1143      Australian Grand Prix   
4       5         1207      Azerbaijan Grand Prix   
5       6         1208           Miami Grand Prix   
6       7         1210          Monaco Grand Prix   
7       8         1211         Spanish Grand Prix   
8       9         1212        Canadian Grand Prix   
9      10         1213        Austrian Grand Prix   
10     11         1214         British Grand Prix   
11     12         1215       Hungarian Grand Prix   
12     13         1216         Belgian Grand Prix   
13     14         1217           Dutch Grand Prix   
14     15         1218         Italian Grand Prix   
15     16         1219       Singapore Grand Prix   
16     17         1220        Japanese Grand Prix   
17     18         1221           Qatar Grand P